In [1]:
!pip install datasets transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.9/452.9 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 65.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 17.1 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [2]:
from transformers import pipeline


In [3]:
from datasets import load_dataset
ag_news = load_dataset("ag_news")

Generating train split:   0%|          | 0/120000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/7600 [00:00<?, ? examples/s]

Dataset ag_news downloaded and prepared to /root/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
print(ag_news)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 120000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 7600
    })
})


In [5]:
ag_news_test = ag_news["test"]
print(ag_news_test)


Dataset({
    features: ['text', 'label'],
    num_rows: 7600
})


In [6]:
ag_news_test.set_format("pandas")

df_ag_news_test = ag_news_test[:]

In [7]:
type(df_ag_news_test)

pandas.core.frame.DataFrame

In [8]:
df_ag_news_test["label_name"] = df_ag_news_test["label"].apply(lambda x: ag_news_test.features["label"].int2str(x))

In [9]:
df_ag_news_test

,text,label,label_name
0,Fears for T N pension after talks Unions repre...,2,Business
1,The Race is On: Second Private Team Sets Launc...,3,Sci/Tech
2,Ky. Company Wins Grant to Study Peptides (AP) ...,3,Sci/Tech
3,Prediction Unit Helps Forecast Wildfires (AP) ...,3,Sci/Tech
4,Calif. Aims to Limit Farm-Related Smog (AP) AP...,3,Sci/Tech
...,...,...,...
7595,Around the world Ukrainian presidential candid...,0,World
7596,Void is filled with Clement With the supply of...,1,Sports
7597,Martinez leaves bitter Like Roger Clemens did ...,1,Sports
7598,5 of arthritis patients in Singapore take Bext...,2,Business


In [11]:
ag_news["train"].set_format("pandas")

df_ag_news_train = ag_news["train"][:]

In [12]:
df_ag_news_train["label"].apply(lambda x: ag_news_test.features["label"].int2str(x)).value_counts()

Business    30000
Sci/Tech    30000
Sports      30000
World       30000
Name: label, dtype: int64

# Tokenization

In [13]:
from transformers import AutoTokenizer
model_name = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [14]:
 

train_tokenized = tokenizer(df_ag_news_train["text"].tolist(), padding=True, truncation=True)
test_tokenized = tokenizer(df_ag_news_test["text"].tolist(), padding=True, truncation=True)

In [15]:
from transformers import AutoModel
import torch
model = AutoModel.from_pretrained(model_name, output_hidden_states = True).to("cuda")

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [16]:
from tqdm import tqdm
def get_embeddings(sentences, batch_size=40, max_length = 64):
 #disable automatic calculation of the gradient - reduces the memory consumption
  with torch.no_grad():
    for idx in tqdm(range(0, len(list(sentences)), batch_size)):
      batched_sentences = sentences[idx : min(len(sentences), idx + batch_size)]
      encoded = tokenizer(batched_sentences, truncation=True, return_tensors="pt",padding="max_length", max_length=max_length).to("cuda")
      #extract embedding of the first [CLS] token
      yield model(**encoded).last_hidden_state[:,0].cpu().numpy()

In [17]:
import torch
class Dataset(torch.utils.data.Dataset):
   def __init__(self, texts, labels=None):
       self.encodings = tokenizer(texts, padding=True, truncation=True, max_length=512)
       self.labels = labels
   def __getitem__(self, idx):
       item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
       if self.labels:
           item["labels"] = torch.tensor(self.labels[idx])
       return item
   def __len__(self):
       return len(self.encodings["input_ids"])

In [ ]:
 

train_dataset = Dataset(ag_news["train"]["text"].tolist(), ag_news["train"]["label"].tolist())
test_dataset = Dataset(ag_news["test"]["text"].tolist(), ag_news["test"]["label"].tolist())

In [1]:
from transformers import AutoModelForSequenceClassification
model = (AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=4).to("cuda"))

NameError: ignored